# CMPSC 497 Final - LLM Fine-Tuning (Cybersecurity Problem → Approach)

In [3]:
!pip install transformers datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.5 MB/s eta 0:00:00


In [4]:
# uploading dataset
from google.colab import files
uploaded = files.upload()

Saving cybersecurity_dataset_10000_unique.jsonl to cybersecurity_dataset_10000_unique.jsonl


In [5]:
# Step 2: Load the cybersecurity dataset
from datasets import load_dataset
data = load_dataset('json', data_files='cybersecurity_dataset_10000_unique.jsonl')['train']
data = data.train_test_split(test_size=0.2)  # 80% train, 20% test
data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['problem', 'approach'],
        num_rows: 1574
    })
    test: Dataset({
        features: ['problem', 'approach'],
        num_rows: 394
    })
})

In [6]:
# Step 3: Load tokenizer and model
from transformers import AutoModelForCausalLM, GPT2Tokenizer

model = AutoModelForCausalLM.from_pretrained('distilgpt2')
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
# Step 4: Tokenize the dataset
def tokenize(batch):
    texts = [p + " " + a for p, a in zip(batch["problem"], batch["approach"])]
    out = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=96
    )
    out["labels"] = [ids.copy() for ids in out["input_ids"]]
    return out

enc_data = data.map(tokenize, batched=True, remove_columns=["problem", "approach"])

Map:   0%|          | 0/1574 [00:00<?, ? examples/s]

Map:   0%|          | 0/394 [00:00<?, ? examples/s]

In [8]:
!pip install datasets


In [9]:
# Step 5: Define training arguments
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=2,
    logging_steps=10,
    logging_dir='./logs',
    report_to='none'
)


In [10]:
# Step 6: Create Trainer instance and train
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=enc_data['train'],
    eval_dataset=enc_data['test']
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.992200
20,1.053200
30,0.659400
40,0.517600
50,0.407400
60,0.382400
70,0.347800
80,0.298300
90,0.236800
100,0.224600


TrainOutput(global_step=3148, training_loss=0.12443747465159115, metrics={'train_runtime': 312.9922, 'train_samples_per_second': 20.116, 'train_steps_per_second': 10.058, 'total_flos': 154230407036928.0, 'train_loss': 0.12443747465159115, 'epoch': 4.0})

In [12]:
import os
import torch
from transformers import CONFIG_NAME, WEIGHTS_NAME

save_dir = "/content/drive/MyDrive/final_llm_model"

# Create the folder if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

model.to("cpu")
state_dict = model.state_dict()
torch.save(state_dict, f"{save_dir}/{WEIGHTS_NAME}")
model.config.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('/content/drive/MyDrive/final_llm_model/tokenizer_config.json',
 '/content/drive/MyDrive/final_llm_model/special_tokens_map.json',
 '/content/drive/MyDrive/final_llm_model/vocab.json',
 '/content/drive/MyDrive/final_llm_model/merges.txt',
 '/content/drive/MyDrive/final_llm_model/added_tokens.json')

In [15]:
!pip install gradio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 165.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.2 MB/s eta 0:00:00


In [16]:
# Step 8: Load the fine-tuned model for testing
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/drive/MyDrive/final_llm_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

In [17]:
# Step 9: Define generation function
def generate_approach(problem: str):
    prompt = problem.strip()
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        no_repeat_ngram_size=2,
        repetition_penalty=1.1,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text[len(prompt):].strip()

In [18]:
# Step 10: Build Gradio interface
iface = gr.Interface(
    fn=generate_approach,
    inputs=gr.Textbox(lines=3, placeholder="Enter your cybersecurity research problem…"),
    outputs=gr.Textbox(lines=8),
    title="🔐 Cybersecurity LLM",
    description="Type a cybersecurity problem and get an approach generated by the fine-tuned model.",
    examples=[
        ["How to detect phishing attacks in email systems?"],
        ["How to secure IoT devices from network attacks?"]
    ]
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7b144afa60e0583314.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Simple Command-Line Interface (CLI) Testing
while True:
    prompt = input("Enter a cybersecurity research problem (or type 'exit' to quit): ")
    if prompt.lower() == "exit":
        break
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        no_repeat_ngram_size=2,
        repetition_penalty=1.1,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = text[len(prompt):].strip()
    print("\nApproach generated by the model:")
    print(answer)
    print("\n")